In [3]:
import spacy
from collections import defaultdict, Counter
import random

nlp = spacy.load("ru_core_news_sm")

In [4]:
file_path = '/home/roman/Downloads/Doner_Loren_Furi_(LP)_Litmir.net_bid224439_2a09f.txt'

with open(file_path, 'r', encoding='cp1251') as f:
    raw_text = f.read()

doc = nlp(raw_text)

tokens = [
    token.lemma_.lower()
    for token in doc
    if token.is_alpha and not token.is_space and not token.is_punct
]

print("Пример первых 20 слов:")
print(" ".join(tokens[:20]))

Пример первых 20 слов:
лорен донер фьюри пролог вот дерьмо пробормотать элли себя под нос наблюдать за мужчина приковать к стена в соседний комната


In [5]:
def make_ngrams(token_list, n):
    return [tuple(token_list[i:i + n]) for i in range(len(token_list) - n + 1)]

bigrams = make_ngrams(tokens, 2)
trigrams = make_ngrams(tokens, 3)

print(f"Биграмм: {len(bigrams):,}")
print(f"Триграмм: {len(trigrams):,}")

def build_ngram_model(ngram_list):
    model = defaultdict(Counter)
    for gram in ngram_list:
        context = gram[:-1]
        next_word = gram[-1]
        model[context][next_word] += 1
    return model

bigram_model = build_ngram_model(bigrams)
trigram_model = build_ngram_model(trigrams)

Биграмм: 99,662
Триграмм: 99,661


In [6]:
from collections import defaultdict, Counter

def build_ngram_model(ngram_list):
    model = defaultdict(Counter)
    
    for gram in ngram_list:
       
        context = gram[:-1]
        next_word = gram[-1]
        model[context][next_word] += 1
    
    return model

bigram_model = build_ngram_model(bigrams)
trigram_model = build_ngram_model(trigrams)

example = ('он',)
print(f"После слова \"{example[0]}\" чаще всего шли:")
for word, count in bigram_model[example].most_common(5):
    print(f" - \"{word}\" — {count} раз")

После слова "он" чаще всего шли:
 - "не" — 121 раз
 - "быть" — 55 раз
 - "хотеть" — 44 раз
 - "лицо" — 41 раз
 - "взгляд" — 38 раз


In [7]:
def generate_text(model, n, length=10):
    context = random.choice(list(model.keys()))
    output = list(context)
    
    for _ in range(length):
        if context not in model:
            break
        next_words = model[context]
        
        next_word = random.choices(
            list(next_words.keys()),
            weights=list(next_words.values())
        )[0]
        
        output.append(next_word)
        context = tuple(output[-(n - 1):])
    
    return " ".join(output)

In [9]:
print("\nBigrams:")
print(generate_text(bigram_model, 2, length=10))
print("\nTrigrams:")
print(generate_text(trigram_model, 3, length=10))


Bigrams:
глянула на весь это не сжечь лифчик фьюри не сделать элли

Trigrams:
бриз оказаться достаточно доказательство чтобы судья наконец то оставить элли в лаборатория






Сейчас позапускав пару раз, смог вывести два текста, первый исходя из биграмм, второй - из триграмм. И отчётливо видно, что в биграммах сгенерился будто бы текст из рандомных слов, оочень странный порядок, во втором же предожении прослеживается некий смысл, даже несмотря на то, что всё лемматизировано. Короче говоря, чем больше контекста знает модель, тем лучше. ЯВНО лучше, даже если контекст всего на одно слово больше.